In [12]:
from api_keys import API_KEY
import requests, json

bills = requests.get(f'https://api.congress.gov/v3/bill/117/s?&api_key={API_KEY}')

In [30]:

bill_nye = bills.json()
print(bill_nye)
len(bill_nye["bills"])
bill_values = []

with open("bill_data.csv","w") as bill_data:
    bills_list = bill_nye["bills"]
    for bill in bills_list:
        bill_number = bill["number"]
        individual_bill = requests.get(f"https://api.congress.gov/v3/bill/117/s/{bill_number}?api_key={API_KEY}")
        bill_dict = individual_bill.json()["bill"]
        for sponsor in bill_dict["sponsors"]:
            bill_values = [bill_dict["number"], sponsor["fullName"], sponsor["state"],bill_dict["policyArea"]["name"]]
            print(bill_values, file=bill_data)


{'bills': [{'congress': 117, 'latestAction': {'actionDate': '2023-01-05', 'text': 'Became Public Law No: 117-335.'}, 'number': '989', 'originChamber': 'Senate', 'originChamberCode': 'S', 'title': 'Native American Language Resource Center Act of 2022', 'type': 'S', 'updateDate': '2023-03-08', 'updateDateIncludingText': '2023-03-08T20:12:22Z', 'url': 'https://api.congress.gov/v3/bill/117/s/989?format=json'}, {'congress': 117, 'latestAction': {'actionDate': '2023-01-05', 'text': 'Became Public Law No: 117-362.'}, 'number': '5329', 'originChamber': 'Senate', 'originChamberCode': 'S', 'title': 'A bill to amend the Bill Emerson Good Samaritan Food Donation Act to improve the program, and for other purposes.', 'type': 'S', 'updateDate': '2023-03-08', 'updateDateIncludingText': '2023-03-08T19:58:35Z', 'url': 'https://api.congress.gov/v3/bill/117/s/5329?format=json'}, {'congress': 117, 'latestAction': {'actionDate': '2023-01-05', 'text': 'Became Public Law No: 117-361.'}, 'number': '5328', 'ori